# End-to-end Vehicles/Pedestrians Segmentation with Labellerr + YOLOv8-seg + ByteTrack

This notebook implements an end-to-end, modular workflow:
- Data collection (Kaggle dataset)
- Annotation using Labellerr SDK (project, upload, export to YOLO-seg)
- Training YOLOv8 segmentation (~100 epochs)
- Inference + evaluation on held-out test set
- Uploading predictions as pre-annotations to a Labellerr test project
- Video tracking with ByteTrack and export to results.json
- Packaging artifacts and a PDF summary

Run cells one-by-one. Cells are designed to be safe on Colab too.

In [1]:
# Section 1: Environment Setup and Project Structure
import os, sys, shutil, json, random
from pathlib import Path

# Colab-friendly dependency installs
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    # Core libs and training/eval stack
    !pip -q install ultralytics opencv-python-headless seaborn matplotlib pandas tqdm reportlab pycocotools supervision
    # Labellerr SDK (optional; only needed if using their API)
    !pip -q install https://github.com/tensormatics/SDKPython/releases/download/prod/labellerr_sdk-1.0.0.tar.gz
    # KaggleHub for dataset download
    !pip -q install kagglehub

BASE_DIR = Path('.')
IMAGES_RAW_DIR = BASE_DIR / 'data' / 'images' / 'raw'
LABELS_DIR = BASE_DIR / 'data' / 'labels'
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
APPS_DIR = BASE_DIR / 'apps'
for p in [IMAGES_RAW_DIR, LABELS_DIR, ARTIFACTS_DIR, APPS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print('Base dir:', BASE_DIR.resolve())
print('Dirs ready:', IMAGES_RAW_DIR, LABELS_DIR, ARTIFACTS_DIR, APPS_DIR)

def ensure_dirs(paths):
    for p in paths:
        Path(p).mkdir(parents=True, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Base dir: /content
Dirs ready: data/images/raw data/labels artifacts apps


## Section 2: Dataset Download (Kaggle)
We will use the Kaggle dataset "ashfakyeafi/road-vehicle-images-dataset" via KaggleHub and stage the images into `data/images/raw` for this notebook.

In [2]:
# Dataset download and staging (Kaggle)
import os, shutil, glob
from pathlib import Path

try:
    import kagglehub as kh
except Exception:
    # If running locally, ensure kagglehub is installed. In Colab, Section 1 installs it.
    !pip -q install kagglehub
    import kagglehub as kh

# OPTIONAL: Kaggle auth only if dataset requires it
# from google.colab import files
# files.upload()  # upload kaggle.json
# Path('~/.kaggle').expanduser().mkdir(parents=True, exist_ok=True)
# shutil.move('kaggle.json', str(Path('~/.kaggle/kaggle.json').expanduser()))
# os.chmod(str(Path('~/.kaggle/kaggle.json').expanduser()), 0o600)

# Download dataset
ds_path = kh.dataset_download("ashfakyeafi/road-vehicle-images-dataset")
print("Path to dataset files:", ds_path)

# Stage images into the notebook's raw images folder
IMAGES_RAW_DIR.mkdir(parents=True, exist_ok=True)

# Basic image validation
import cv2

def is_valid_image(path: str) -> bool:
    try:
        img = cv2.imread(path)
        return img is not None
    except Exception:
        return False

# Collect common image types from the downloaded dataset
img_exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
found = []
for pat in img_exts:
    found.extend(glob.glob(str(Path(ds_path) / "**" / pat), recursive=True))

# Copy (skip existing) into data/images/raw
copied = 0
existing_names = {p.name for p in IMAGES_RAW_DIR.glob("*")}
for src in found:
    src_p = Path(src)
    if src_p.name in existing_names:
        continue
    if is_valid_image(str(src_p)):
        shutil.copy2(src_p, IMAGES_RAW_DIR / src_p.name)
        copied += 1

total = len(list(IMAGES_RAW_DIR.glob("*")))
print(f"Copied {copied} new images. Total in {IMAGES_RAW_DIR}: {total}")

100%|██████████| 115M/115M [00:00<00:00, 182MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ashfakyeafi/road-vehicle-images-dataset/versions/2
Copied 3004 new images. Total in data/images/raw: 3004


In [3]:
# Build sources.md from images present in data/images/raw
import pandas as pd
from pathlib import Path


def build_sources_md_from_dir(images_dir: Path, out_md: str = 'data/sources.md'):
    images = sorted([str(p) for p in Path(images_dir).glob('*.jpg')])
    lines = ["# Image Sources and Licenses\n"]
    for p in images:
        # Source unknown because of Kaggle packaging; update if you have per-file metadata
        lines.append(f"- file: {p}, url: unknown (Kaggle package), license: check dataset license")
    Path(out_md).parent.mkdir(parents=True, exist_ok=True)
    Path(out_md).write_text("\n".join(lines), encoding='utf-8')
    return out_md

sources_md_path = build_sources_md_from_dir(IMAGES_RAW_DIR)
print('sources.md at', sources_md_path)

sources.md at data/sources.md


## Section 3: Labellerr SDK Auth and Project Creation
Initialize the Labellerr SDK client and create a training project with an ontology for person and vehicle polygon masks.

In [14]:
# Labellerr client and project creation helpers
import os

# Install in Colab if needed:
# !pip -q install https://github.com/tensormatics/SDKPython/releases/download/prod/labellerr_sdk-1.0.0.tar.gz

try:
    from labellerr.client import LabellerrClient  # type: ignore
except Exception as e:
    LabellerrClient = None
    print('Labellerr SDK not installed. Install from docs if needed.')


def labellerr_client():
    if LabellerrClient is None:
        return None
    api_key = os.getenv('LABELLERR_API_KEY','8d24e9.bc324a450a90c026b763a2ae8a')
    api_secret = os.getenv('LABELLERR_API_SECRET','5b7dda285f37211090a4ac04e1c3c0dfdf7152e110070ad487947a159ce6aff1')
    if not api_key or not api_secret:
        print('Set LABELLERR_API_KEY and LABELLERR_API_SECRET in env')
        return None
    return LabellerrClient(api_key, api_secret)


def labellerr_create_project(client, name='Vehicles-Pedestrians-SEG', desc='Training segmentation project', ontology=None):
    if client is None:
        return None
    # Placeholder; exact SDK calls depend on Labellerr SDK
    try:
        # Example skeleton (adjust per SDK):
        project = client.create_project({
            'name': name,
            'description': desc,
            'ontology': ontology or {
                'tools': ['polygon'],
                'classes': ['person', 'vehicle']
            },
            'data_type': 'image', # Replace with your data type
            'client_id': 'your_client_id', # Replace with your client ID
            'dataset_id': 'your_dataset_id', # Replace with your dataset ID
            'annotation_template_id': 'your_template_id', # Replace with your template ID
            'rotation_config': {} # Replace with your rotation config
        })
        print('Created project:', project)
        return project.get('id') if isinstance(project, dict) else project
    except Exception as e:
        print('Project creation error:', e)
        return None

client = labellerr_client()
# Example call:
train_project_id = labellerr_create_project(client)
print('Train project id:', train_project_id)

Project creation error: LabellerrClient.create_project() missing 5 required positional arguments: 'data_type', 'client_id', 'dataset_id', 'annotation_template_id', and 'rotation_config'
Train project id: None


## Section 5: Upload Images and Create Polygon Segmentation Task
Upload up to 100 images and create an annotation task. This section includes polling until completion.

In [5]:
# Upload images and create annotation task (SDK placeholders)
import time
from pathlib import Path

def labellerr_upload_images(client, project_id, image_dir, limit=100):
    if client is None or not project_id:
        print('Client or project missing')
        return []
    imgs = sorted([str(p) for p in Path(image_dir).glob('*.jpg')])[:limit]
    uploaded_ids = []
    for p in imgs:
        try:
            # Placeholder for actual SDK upload
            r = client.upload_image(project_id=project_id, image_path=p)
            uploaded_ids.append(r.get('id') if isinstance(r, dict) else r)
        except Exception as e:
            print('Upload failed for', p, e)
    print('Uploaded', len(uploaded_ids), 'images')
    return uploaded_ids


def labellerr_create_annotation_task(client, project_id, assignees=None, instructions='Annotate vehicles and pedestrians with polygon masks'):
    if client is None or not project_id:
        return None
    try:
        task = client.create_task({
            'project_id': project_id,
            'type': 'segmentation',
            'tool': 'polygon',
            'instructions': instructions,
            'assignees': assignees or [],
        })
        print('Created task:', task)
        return task
    except Exception as e:
        print('Create task error:', e)
        return None


def poll_task_until_complete(client, task_id, timeout=7200, interval=20):
    start = time.time()
    while time.time() - start < timeout:
        try:
            status = client.get_task_status(task_id)
            print('Task status:', status)
            if status in ['completed','done','exported']:
                return status
        except Exception as e:
            print('Poll error:', e)
        time.sleep(interval)
    return 'timeout'

# Example calls (commented to avoid accidental execution without credentials)
uploaded = labellerr_upload_images(client, train_project_id, IMAGES_RAW_DIR)
# task = labellerr_create_annotation_task(client, train_project_id)
# poll_task_until_complete(client, task.get('id'))

## Section 6: Export Annotations from Labellerr in YOLO-Seg Format
Export labels in YOLO segmentation format and validate pairs.

In [6]:
# Export and validate YOLO-seg labels (SDK placeholders)
import zipfile

def labellerr_export_yolo_seg(client, project_id, out_dir='data/labels'):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    if client is None or not project_id:
        print('Client/project missing')
        return None
    try:
        # Placeholder for actual export call
        zpath = client.export_annotations(project_id=project_id, format='yolo-seg')  # returns zip path or URL
        # If URL, download to out_dir; here assume local path
        dst = Path(out_dir) / 'labels_export.zip'
        shutil.copy(zpath, dst)
        with zipfile.ZipFile(dst, 'r') as zf:
            zf.extractall(out_dir)
        print('Exported to', out_dir)
        return out_dir
    except Exception as e:
        print('Export error:', e)
        return None


def validate_pairs(images_dir, labels_dir):
    images = {p.stem for p in Path(images_dir).glob('*.jpg')}
    labels = {p.stem for p in Path(labels_dir).glob('*.txt')}
    missing = images - labels
    if missing:
        print('Missing labels for images:', list(sorted(missing))[:10], '...')
    else:
        print('All image-label pairs present.')

# Example
# export_dir = labellerr_export_yolo_seg(client, train_project_id, out_dir=str(LABELS_DIR))
# validate_pairs(IMAGES_RAW_DIR, LABELS_DIR)

## Section 7: Prepare YOLO Data Folders and YAML Splits
Create train/val/test splits and generate a `data.yaml` for YOLOv8-seg.

In [7]:
# Splitting and data.yaml helpers
import random, yaml


def make_splits(images_dir, labels_dir, out_base='data', ratios=(0.7,0.2,0.1), seed=42):
    images = sorted([p for p in Path(images_dir).glob('*.jpg')])
    random.Random(seed).shuffle(images)
    n = len(images)
    n_tr = int(ratios[0]*n)
    n_va = int(ratios[1]*n)
    parts = {
        'train': images[:n_tr],
        'val': images[n_tr:n_tr+n_va],
        'test': images[n_tr+n_va:]
    }
    for split, im_list in parts.items():
        img_out = Path(out_base)/'images'/split
        lbl_out = Path(out_base)/'labels'/split
        img_out.mkdir(parents=True, exist_ok=True)
        lbl_out.mkdir(parents=True, exist_ok=True)
        for ip in im_list:
            lp = Path(labels_dir)/ (ip.stem + '.txt')
            if lp.exists():
                shutil.copy(ip, img_out/ip.name)
                shutil.copy(lp, lbl_out/lp.name)
    return parts


def write_data_yaml(path='data/data.yaml', names=('person','vehicle')):
    data = {
        'path': 'data',
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': list(names),
        'nc': len(names)
    }
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        yaml.safe_dump(data, f)
    return path

# Example once labels are present:
# parts = make_splits(IMAGES_RAW_DIR, LABELS_DIR)
# data_yaml = write_data_yaml()
# print('Data YAML at', data_yaml)

## Section 8: Train YOLOv8-seg (~100 epochs) and Save Metrics
Train using Ultralytics and collect metrics and plots.

In [8]:
# Training helper
from ultralytics import YOLO
import pandas as pd


def train_yolov8_seg(model='yolov8n-seg.pt', data='data/data.yaml', epochs=100, imgsz=640, project='artifacts/runs', name='train-seg'):
    Path(project).mkdir(parents=True, exist_ok=True)
    m = YOLO(model)
    r = m.train(data=data, epochs=epochs, imgsz=imgsz, project=project, name=name, save=True, plots=True)
    run_dir = Path(r.save_dir)
    # Copy key plots
    metrics_dir = Path('artifacts/metrics')
    metrics_dir.mkdir(parents=True, exist_ok=True)
    for plot_name in ['F1_curve.png','PR_curve.png','R_curve.png','P_curve.png','confusion_matrix.png']:
        src = run_dir/plot_name
        if src.exists():
            shutil.copy(src, metrics_dir/src.name)
    # Results CSV
    results_csv = run_dir/'results.csv'
    if results_csv.exists():
        df = pd.read_csv(results_csv)
        df.to_csv(metrics_dir/'results.csv', index=False)
    return str(run_dir)

# Example:
run_dir = train_yolov8_seg(epochs=100)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Section 9: Evaluate on Test Set and Plot Metrics
Validate the trained model and save metrics/plots.

In [9]:
# Evaluation helper
import seaborn as sns
import matplotlib.pyplot as plt


def evaluate_model(weights, data_yaml='data/data.yaml', split='test'):
    m = YOLO(weights)
    r = m.val(data=data_yaml, split=split, save_json=True, plots=True)
    out_dir = Path(r.save_dir)
    metrics_dir = Path('artifacts/metrics')
    metrics_dir.mkdir(parents=True, exist_ok=True)
    # Copy plots/metrics
    for p in out_dir.glob('*.png'):
        shutil.copy(p, metrics_dir/p.name)
    for p in out_dir.glob('*.json'):
        shutil.copy(p, metrics_dir/p.name)
    print('Metrics saved to artifacts/metrics')
    return str(out_dir)

# Example:
# eval_dir = evaluate_model(weights='artifacts/runs/train-seg/weights/best.pt')

## Section 10: Inference on Unseen Images and Save predictions.json
Run inference and save predictions (boxes, classes, masks as polygons if available).

In [10]:
# Inference helper
from shapely.geometry import Polygon


def masks_to_polygons(masks):
    polys = []
    if masks is None:
        return polys
    try:
        for m in masks.xy:  # Ultralytics provides polygons for seg models
            # m is list of [x,y] points
            pts = [float(v) for v in m.reshape(-1).tolist()]
            polys.append(pts)
    except Exception:
        pass
    return polys


def infer_and_save_json(weights, source='data/images/test', out_json='artifacts/predictions.json', conf=0.25, iou=0.7, imgsz=640):
    m = YOLO(weights)
    results = m(source=source, conf=conf, iou=iou, imgsz=imgsz, save=False, stream=True, verbose=False)
    preds = []
    for r in results:
        im_path = getattr(r, 'path', None)
        names = r.names
        if r.boxes is not None:
            boxes = r.boxes.xyxy.cpu().numpy().tolist()
            cls = r.boxes.cls.int().cpu().numpy().tolist()
            confs = r.boxes.conf.cpu().numpy().tolist()
            polys = masks_to_polygons(getattr(r, 'masks', None))
            for i, (b, c, s) in enumerate(zip(boxes, cls, confs)):
                item = {
                    'image_path': im_path,
                    'cls': int(c),
                    'cls_name': names.get(int(c), str(c)) if isinstance(names, dict) else str(c),
                    'conf': float(s),
                    'bbox_xyxy': [float(v) for v in b],
                }
                if i < len(polys):
                    item['mask_polygon'] = polys[i]
                preds.append(item)
    Path(out_json).parent.mkdir(parents=True, exist_ok=True)
    with open(out_json, 'w') as f:
        json.dump(preds, f, indent=2)
    print('Saved predictions to', out_json)
    return out_json

# Example:
# pred_json = infer_and_save_json('artifacts/runs/train-seg/weights/best.pt')

## Section 11: Labellerr Test Project and Pre-annotations Upload
Create a test project and upload predictions as pre-annotations.

In [11]:
# Pre-annotation upload (SDK placeholders)

def labellerr_upload_preannotations(client, project_id, predictions_json):
    if client is None or not project_id:
        print('Client/project missing')
        return False
    data = json.load(open(predictions_json,'r'))
    # Convert to expected payload; structure depends on SDK specifics.
    payload = []
    for item in data:
        payload.append({
            'image_path': item['image_path'],
            'annotations': [{
                'class': item['cls_name'],
                'bbox_xyxy': item['bbox_xyxy'],
                'polygon': item.get('mask_polygon')
            }]
        })
    try:
        resp = client.upload_preannotations(project_id=project_id, annotations=payload)
        print('Pre-annotations upload response:', resp)
        return True
    except Exception as e:
        print('Pre-annotations upload error:', e)
        return False

# Example:
# test_project_id = labellerr_create_project(client, name='Vehicles-Pedestrians-SEG-Test', desc='Test project')
# ok = labellerr_upload_preannotations(client, test_project_id, 'artifacts/predictions.json')

## Section 12: Video Tracking with YOLOv8 + ByteTrack
Use Ultralytics' built-in ByteTrack integration to track vehicles and pedestrians in videos.

In [12]:
# Tracking helper using Ultralytics' ByteTrack

def track_video(weights, video_path, out_dir='artifacts/tracking', tracker='bytetrack.yaml', imgsz=640, conf=0.25):
    from ultralytics import YOLO
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    m = YOLO(weights)
    res = m.track(source=video_path, tracker=tracker, imgsz=imgz, conf=conf, save=True, persist=True, project=out_dir, name='track_run')
    # Build results.json
    results_json = []
    for r in res:
        if r.boxes is None or r.boxes.id is None:
            continue
        ids = r.boxes.id.int().cpu().tolist()
        xyxy = r.boxes.xyxy.cpu().numpy().tolist()
        cls = r.boxes.cls.int().cpu().tolist()
        names = r.names
        frame_idx = getattr(r, 'frame', None)
        for tid, box, c in zip(ids, xyxy, cls):
            results_json.append({'frame': int(frame_idx) if frame_idx is not None else -1,
                                 'track_id': int(tid), 'class_id': int(c), 'class_name': names.get(int(c), str(c)) if isinstance(names, dict) else str(c),
                                 'bbox_xyxy': [float(v) for v in box]})
    out_json = Path(out_dir)/'results.json'
    with open(out_json, 'w') as f:
        json.dump(results_json, f, indent=2)
    print('Tracking results saved to', out_json)
    return str(out_json)

# Example:
# results_json = track_video(weights='yolov8n.pt', video_path='sample.mp4')

## Section 13: Streamlit App Wrapper
Use the provided app at `apps/tracker_app.py` and run it locally to test tracking UI.

In [ ]:
# To run locally (PowerShell):
# python -m venv .venv; .\.venv\Scripts\Activate.ps1
# pip install -r requirements.txt
# streamlit run apps/tracker_app.py
print('Tracker app path:', (BASE_DIR/'apps'/'tracker_app.py').resolve())

## Section 14: Artifact Packaging (README and PDF Summary)
Generate a README and a PDF summary report from metrics and examples.

In [ ]:
# README and PDF summary
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

def write_readme(path='artifacts/README.md', context=None):
    lines = [
        '# Results and Notes',
        '',
        'This folder contains metrics, plots, and example outputs.',
        'Refer to root README.md for setup and instructions.',
    ]
    if context:
        lines += ['', '## Context', json.dumps(context, indent=2)]
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    Path(path).write_text('\n'.join(lines), encoding='utf-8')
    return path


def export_pdf_summary(out_pdf='artifacts/summary.pdf', metrics_dir='artifacts/metrics'):
    Path(out_pdf).parent.mkdir(parents=True, exist_ok=True)
    c = canvas.Canvas(out_pdf, pagesize=A4)
    w, h = A4
    y = h - 50
    c.setFont('Helvetica-Bold', 16)
    c.drawString(50, y, 'Labellerr + YOLOv8-seg + ByteTrack Summary')
    y -= 30
    c.setFont('Helvetica', 12)
    c.drawString(50, y, 'Key metrics and plots')
    y -= 20
    # Embed a couple of plots if present
    for img_name in ['PR_curve.png','confusion_matrix.png','F1_curve.png']:
        p = Path(metrics_dir)/img_name
        if p.exists():
            y -= 300
            c.drawImage(str(p), 50, y, width=500, height=280, preserveAspectRatio=True, anchor='nw')
            y -= 20
            c.drawString(50, y, img_name)
            y -= 10
    c.showPage()
    c.save()
    print('PDF summary at', out_pdf)
    return out_pdf

# Example:
# readme_path = write_readme()
# pdf_path = export_pdf_summary()